In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
train_df = pd.read_csv("./AIFrenz_Season1_dataset/train.csv")
train_df.drop(['X14','X16','X19'], axis=1, inplace=True)
print(train_df.shape)
train_df.head()

(4752, 57)


,id,X00,X01,X02,X03,X04,X05,X06,X07,X08,...,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
0,0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,...,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,NaN
1,1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,...,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,NaN
2,2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,...,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,NaN
3,3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,...,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,NaN
4,4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,...,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,NaN


In [3]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [4]:
input_part = np.array(train_df[train_df.columns[1:38]].iloc[4320:], dtype=np.float32)
target_part = np.array(train_df[train_df.columns[-1:]].iloc[4320:], dtype=np.float32).reshape(-1)

In [5]:
first=True
for idx in range(len(input_part)-1):
    diff_x = input_part[idx+1] - input_part[idx]
    diff_y = target_part[idx+1] - target_part[idx]
    if first:
        x = diff_x.reshape(1,-1)
        y = [diff_y]
        first=False
    else:
        x = np.concatenate((x, diff_x.reshape(1,-1)), axis=0)
        y.append(diff_y)
y = np.array(y, dtype=np.float32)

In [6]:
train_input = x[:144*2-1]
train_target = y[:144*2-1]
val_input = x[144*2-1:]
val_target = y[144*2-1:]

In [7]:
print(train_input.shape, train_target.shape, val_input.shape, val_target.shape)

(287, 37) (287,) (144, 37) (144,)


In [8]:
global_min = train_input.min(axis=0)
global_max = train_input.max(axis=0)
global_range = global_max-global_min
global_range = np.where(global_range==0,1,global_range)
train_input = (train_input-global_min)/global_range
val_input = (val_input-global_min)/global_range
print("============================================= Shape is =============================================")
print(train_input.shape, train_target.shape, val_input.shape, val_target.shape)

============================================= Shape is =============================================
(287, 37) (287,) (144, 37) (144,)


In [9]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    #'num_leaves': 5,
    'learning_rate': 0.01,
    'metric' : ['l1','l2'],
    'random_state': 501,
}
lgb_train = lgb.Dataset(train_input, train_target)
lgb_eval = lgb.Dataset(val_input, val_target)

gbm = lgb.train(params, lgb_train, num_boost_round=20000,valid_sets=[lgb_eval, lgb_train],early_stopping_rounds=700)

[1]	training's l1: 0.377108	training's l2: 0.398618	valid_0's l1: 0.365948	valid_0's l2: 0.344224
Training until validation scores don't improve for 700 rounds
[2]	training's l1: 0.376453	training's l2: 0.396592	valid_0's l1: 0.365783	valid_0's l2: 0.34296
[3]	training's l1: 0.375933	training's l2: 0.394606	valid_0's l1: 0.365778	valid_0's l2: 0.34173
[4]	training's l1: 0.375456	training's l2: 0.39266	valid_0's l1: 0.365795	valid_0's l2: 0.340527
[5]	training's l1: 0.375149	training's l2: 0.390752	valid_0's l1: 0.366003	valid_0's l2: 0.339361
[6]	training's l1: 0.374844	training's l2: 0.38888	valid_0's l1: 0.366227	valid_0's l2: 0.338266
[7]	training's l1: 0.374601	training's l2: 0.387046	valid_0's l1: 0.366432	valid_0's l2: 0.337156
[8]	training's l1: 0.374351	training's l2: 0.385247	valid_0's l1: 0.366667	valid_0's l2: 0.33608
[9]	training's l1: 0.374063	training's l2: 0.383483	valid_0's l1: 0.3669	valid_0's l2: 0.335073
[10]	training's l1: 0.373797	training's l2: 0.381754	valid_0's 

[238]	training's l1: 0.265661	training's l2: 0.172568	valid_0's l1: 0.381334	valid_0's l2: 0.280718
[239]	training's l1: 0.265323	training's l2: 0.172071	valid_0's l1: 0.381498	valid_0's l2: 0.280741
[240]	training's l1: 0.264868	training's l2: 0.171552	valid_0's l1: 0.381491	valid_0's l2: 0.280762
[241]	training's l1: 0.26439	training's l2: 0.170947	valid_0's l1: 0.381581	valid_0's l2: 0.280731
[242]	training's l1: 0.264012	training's l2: 0.170462	valid_0's l1: 0.38154	valid_0's l2: 0.280757
[243]	training's l1: 0.263583	training's l2: 0.169795	valid_0's l1: 0.381823	valid_0's l2: 0.280846
[244]	training's l1: 0.263239	training's l2: 0.169311	valid_0's l1: 0.381683	valid_0's l2: 0.280646
[245]	training's l1: 0.263044	training's l2: 0.168956	valid_0's l1: 0.382068	valid_0's l2: 0.280799
[246]	training's l1: 0.262537	training's l2: 0.168318	valid_0's l1: 0.382081	valid_0's l2: 0.280817
[247]	training's l1: 0.26211	training's l2: 0.167669	valid_0's l1: 0.382359	valid_0's l2: 0.280914
[24

[523]	training's l1: 0.187794	training's l2: 0.0843443	valid_0's l1: 0.395202	valid_0's l2: 0.28955
[524]	training's l1: 0.187584	training's l2: 0.084166	valid_0's l1: 0.395185	valid_0's l2: 0.289528
[525]	training's l1: 0.187387	training's l2: 0.0839634	valid_0's l1: 0.395262	valid_0's l2: 0.28963
[526]	training's l1: 0.186996	training's l2: 0.0836527	valid_0's l1: 0.395285	valid_0's l2: 0.289551
[527]	training's l1: 0.186894	training's l2: 0.0834908	valid_0's l1: 0.395211	valid_0's l2: 0.289513
[528]	training's l1: 0.186653	training's l2: 0.083286	valid_0's l1: 0.395077	valid_0's l2: 0.289486
[529]	training's l1: 0.18642	training's l2: 0.0831106	valid_0's l1: 0.395202	valid_0's l2: 0.289668
[530]	training's l1: 0.186186	training's l2: 0.0828987	valid_0's l1: 0.395072	valid_0's l2: 0.289597
[531]	training's l1: 0.185903	training's l2: 0.0826506	valid_0's l1: 0.395134	valid_0's l2: 0.289673
[532]	training's l1: 0.185597	training's l2: 0.0824113	valid_0's l1: 0.395272	valid_0's l2: 0.28

In [10]:
pred = gbm.predict(val_input)
target_part = np.array(train_df[train_df.columns[-1:]].iloc[-145], dtype=np.float32).reshape(-1)

In [11]:
pred_trans=[]
first=True
for diff in pred:
    if first:
        pred_trans.append(target_part[0]+diff)
        first=False
    else:
        before = pred_trans[-1]
        pred_trans.append(before+diff)

In [12]:
real = np.array(train_df[train_df.columns[-1:]].iloc[-144:], dtype=np.float32).reshape(-1)
print("Result Is ======================")
print("MSE ==")
print(mean_squared_error(real, pred_trans))
print("MAE ==")
print(mean_absolute_error(real, pred_trans))

Result Is ======================
MSE ==
69.44308700196952
MAE ==
5.89422374564548


In [18]:
from sklearn.svm import SVR

clf = SVR(C=2)
clf.fit(train_input, train_target)
pred = clf.predict(val_input)
target_part = np.array(train_df[train_df.columns[-1:]].iloc[-145], dtype=np.float32).reshape(-1)
pred_trans=[]
first=True
for diff in pred:
    if first:
        pred_trans.append(target_part[0]+diff)
        first=False
    else:
        before = pred_trans[-1]
        pred_trans.append(before+diff)
        
real = np.array(train_df[train_df.columns[-1:]].iloc[-144:], dtype=np.float32).reshape(-1)
print("Result Is ======================")
print("MSE ==")
print(mean_squared_error(real, pred_trans))
print("MAE ==")
print(mean_absolute_error(real, pred_trans))


pred = clf.predict(train_input)
target_part = np.array(train_df[train_df.columns[-1:]].iloc[-432], dtype=np.float32).reshape(-1)
pred_trans=[]
first=True
for diff in pred:
    if first:
        pred_trans.append(target_part[0]+diff)
        first=False
    else:
        before = pred_trans[-1]
        pred_trans.append(before+diff)
real = np.array(train_df[train_df.columns[-1:]].iloc[-431:-144], dtype=np.float32).reshape(-1)
print("Train Result Is ======================")
print("MSE ==")
print(mean_squared_error(real, pred_trans))
print("MAE ==")
print(mean_absolute_error(real, pred_trans))

Result Is ======================
MSE ==
24.917177842711055
MAE ==
3.752615207410364
Train Result Is ======================
MSE ==
24.67829501834971
MAE ==
3.966809056711414


In [8]:
sub_df = pd.read_csv("./AIFrenz_Season1_dataset/test.csv")
sub_df.drop(['X14','X16','X19'], axis=1, inplace=True)
print(sub_df.shape)

(11520, 38)


In [9]:
feed_for_sub = np.array(sub_df[sub_df.columns[1:]])
feed_for_sub = (feed_for_sub-global_mean)/global_std
print(feed_for_sub.shape)

(11520, 37)


/Users/jaemin/opt/anaconda3/envs/py36t2/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
/Users/jaemin/opt/anaconda3/envs/py36t2/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [10]:
sub_pred = gbm.predict(feed_for_sub, num_iteration=gbm.best_iteration)

In [11]:
sub_idx_df = pd.read_csv("./AIFrenz_Season1_dataset/sample_submission.csv")
sub_idx_df["Y18"] = sub_pred
sub_idx_df.head()

,id,Y18
0,4752,20.895868
1,4753,20.863322
2,4754,20.956668
3,4755,21.052310
4,4756,20.903975


In [13]:
sub_idx_df.to_csv("./submit/sub_v05.csv",index=False)